2. Normal Class vs @dataclass vs BaseModel (Pydantic)

In [2]:
#Normal Class Definition
#But — no type checking, no default serialization, and if someone passes age="thirty"… no errors!

class User:
    def __init__(self, name, age):
        self.name = name
        self.age = age

u = User("Alice", 30)

In [ ]:
#Python 3.7 introduced @dataclass to reduce boilerplate.
#But still: no automatic type validation, no built-in serialization.

from dataclasses import dataclass

@dataclass
class User:
    name: str
    age: int

u = User("Alice", 30)
print(u)

In [ ]:
#pydantic.BaseModel
#Clean 
#Type-safe (throws error if age="thirty")
#Easily serializable (.dict(), .json())
#Auto-casting (age="30" will be converted to int)

from pydantic import BaseModel

class User(BaseModel):
    name: str
    age: int

u = User(name="Alice", age=30)
print(u)

3. Type Hinting in Python — The Foundation

In [ ]:
#Before Pydantic can do its magic, it needs a map — 
# and that’s what type hints are. They tell Pydantic (and your IDE!) what types of data to expect.

from typing import Optional, List

name: str         # expects a string
age: int          # expects an integer
score: float      # expects a float
is_active: bool   # expects True or False
bio: Optional[str]        # can be a string or None
tags: List[str]           # a list of strings

4. Your First Pydantic Model

In [7]:
from pydantic import BaseModel

class User(BaseModel):
    name: str
    age: int

u = User(name="Alice", age=30)
print(u)
print(u.model_dump())

name='Alice' age=30
{'name': 'Alice', 'age': 30}


In [ ]:
"""It prints a clean object and a serializable dictionary — 
perfect for APIs or AI tools that exchange JSON."""

User(name="Alice", age="30") # Pydantic automatically casts types when it can.

User(name='Alice', age=30)

In [ ]:
User(name="Alice", age="thirty") #You’ll get a helpful ValidationError. That’s defensive programming done right.

5. Validation Made Easy With Fields

In [ ]:
"""Pydantic’s Field() lets you:
Enforce basic rules like min/max values
Set defaults
Add helpful descriptions
Define regex patterns"""

In [ ]:
from pydantic import BaseModel, Field

#Example 1: User Input Constraints

class User(BaseModel):
    username: str = Field(..., min_length=3, max_length=15, description="Username must be between 3 and 15 characters")
    age: int = Field(..., gt=0, lt=120, description="Age must be between 1 and 119")

In [ ]:
#Example 2: Feedback Form

from typing import Optional
class Feedback(BaseModel):
    rating: int = Field(..., ge=1, le=5, description="Rating from 1 (worst) to 5 (best)")
    comment: Optional[str] = Field(None, max_length=200)

6. Nested Models and Reusability

In [13]:
#With Pydantic, you can compose models like LEGO blocks:

class Address(BaseModel):
    city: str
    zip_code: str

class User(BaseModel):
    name: str
    address: Address

In [17]:
#It automatically parses and validates nested dictionaries. You can even reuse Address in multiple models.

user = User(name="Alice", address={"city": "NYC", "zip_code": "10001"})
print(user)

name='Alice' address=Address(city='NYC', zip_code='10001')


7. Pydantic with FastAPI

In [ ]:
"""FastAPI is one of the most popular Python web frameworks right now — 
and Pydantic is the engine behind its speed and reliability. 
Every time you define a request body or response model in FastAPI, you’re using Pydantic behind the scenes."""

In [ ]:
%pip install fastapi
from fastapi import FastAPI
from pydantic import BaseModel

app = FastAPI()

class Item(BaseModel):
    name: str
    price: float

@app.post("/items/")
async def create_item(item: Item):
    return {"message": "Item received!", "data": item}

In [ ]:
"""FastAPI will:
Automatically parse and validate the request body as an Item
Generate interactive docs (/docs) using the schema from Pydantic
Return a clean JSON response — also powered by Pydantic"""

8. Handy Pydantic Utilities

In [ ]:
"""Pydantic isn’t just about validation — 
it also gives you clean tools to work with structured data. 
These come in handy when converting between JSON, dictionaries, 
or even untrusted inputs (hello, LLMs)."""